# CVI620 Assignment 01

## Academic Integrity Declaration:

We, `Group 2`, (Mimi Dang, Peter Wan, Aryan Khurana, Jeremy Lee), declare that the attached assignment is our own work in accordance with the Seneca Academic Policy.  We have not copied any part of this assignment, manually or electronically, from any other source including web sites, unless specified as references. We have not distributed our work to other students.

## Part 1: A photo booth application:

In [5]:
# Part 1: A photo booth application:

import os
import cv2 as cv

folder_name = 'part1'

# Create a directory to save images if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Use the cv library to access the webcam
webcam = cv.VideoCapture(0)

image_counter = 1

# Reusable styling for rectangles
rect_thickness = 4
rect_line_type = cv.LINE_4

# Reusable styling for text
font_face = cv.FONT_HERSHEY_SIMPLEX
font_scale = 1
text_color = (0, 255, 0)
text_thickness = 2

# Text properties
text = "Press Q to stop recording."
text_2 = "Press S to save an image."

# Set up a named window with a fixed size
cv.namedWindow('Webcam Video', cv.WINDOW_NORMAL)
cv.resizeWindow('Webcam Video', 800, 600)  # Set the desired window size

while True:

    # Get the tuple from webcam.read() stating if the frame was captured and the frame returned
    is_frame_captured, frame = webcam.read()

    # Exit the loop if the video frame wasn't captured
    if not is_frame_captured:
        break

    # Get the frame's dimensions
    frame_height, frame_width = frame.shape[:2] # use the split operator to get the height and width of the frame

    # Dynamically calculate the two texts' positions
    text_position = (10, int(frame_height * 0.08))   # y position starts at 8% of the frame's height
    text_2_position = (10, int(frame_height * 0.20)) # y position starts at 20% of the frame's height

    # Get the size of the text (width, height) and the baseline
    # The baseline is the y-coordinate at the bottom of the text, excluding any descender (e.g., the bottom of letters like 'g' or 'y').
    # Since the baseline isn't needed in this case, we use '_' to ignore it.
    (text_width, text_height), _ = cv.getTextSize(text, font_face, font_scale, text_thickness)
    (text_2_width, text_2_height), _ = cv.getTextSize(text_2, font_face, font_scale, text_thickness)

    # Check if the first text exceeds the frame height, and adjust its position if necessary.
    if text_position[1] + text_height > frame_height:
        text_position = (text_position[0], frame_height - text_height)

    # Check if the second text exceeds the frame height, and adjust its position if necessary.
    if text_2_position[1] + text_2_height > frame_height:
        text_2_position = (text_2_position[0], frame_height - text_2_height)

    # Add the text to the frame
    image01 = cv.putText(frame, text, text_position, font_face, font_scale, text_color, text_thickness, rect_line_type)
    image02 = cv.putText(image01, text_2, text_2_position, font_face, font_scale, text_color, text_thickness, rect_line_type)

    # Show the video frame
    cv.imshow('Webcam Video', image02)

    # Wait for a key press
    key = cv.waitKey(1)

    # Press 's' to save
    if key & 0xff == ord('s'):
        # Define what the image's name should be
        image_filename = f'{folder_name}/image{image_counter}.jpg'

        # Save the current frame as an image with the image_filename (this overwrites any existing image)
        cv.imwrite(image_filename, image02)

        # Log which image was saved
        print(f"Saved {image_filename}")

        # Increase the image count
        image_counter += 1

    # Press 'q' to quit
    elif key & 0xff == ord('q'):
        break

# Release the webcam and destroy all windows
webcam.release()
cv.destroyAllWindows()


## Part 2: Image Arithmetic

### Part 2a: Brightness & Contrast

In [6]:
### Part 2a: Brightness & Contrast
import cv2 as cv
import numpy as np

folder_name = 'part2'
image01_path = f'{folder_name}/image01.png'
image01 = cv.imread(image01_path)

# Check if the image was loaded successfully
if image01 is None:
    print("Error: Could not load image.")
else:
    # Display the original image
    cv.imshow('Original Image', image01)
    
    # Increase brightness by adding a constant (150) to all color channels
    brightness_value = 150

    # Create a matrix of the same shape as the image, filled with the brightness value, 
    # then add it to the original image, effectively increasing brightness.
    bright_image = cv.add(image01, np.ones(image01.shape, dtype='uint8') * brightness_value)  # Add to B, G, R channels. Do not add anything to transparency
    cv.imshow('Brightened Image', bright_image)  # Display the brightened image

    # Change contrast by multiplying the image by a constant (0.5)
    contrast_value = 0.5

    # Adjust contrast by scaling pixel values. A value < 1 decreases contrast, while > 1 increases it.
    # Multiplying the original image by 0.5 reduces the contrast.
    contrast_image = cv.multiply(image01, np.uint8(np.ones(image01.shape)), scale=contrast_value)
    cv.imshow('Contrast Image', contrast_image)  # Display the contrast-adjusted image

    # Wait for a key press and close all windows
    cv.waitKey(0)
    cv.destroyAllWindows()


### Part 2b: Linear blend

In [6]:
### Part 2b: Linear blend
import cv2 as cv

folder_name = 'part2'
image01_path = f'{folder_name}/image01.png'
image02_path = f'{folder_name}/image02.png'
image01 = cv.imread(image01_path)
image02 = cv.imread(image02_path)

# Get the dimensions of the first image so we can resize the second image
height = image01.shape[0]
width = image01.shape[1]

image02 = cv.imread(image02_path)

# Resize the second image to match the first image dimensions
image02 = cv.resize(image02, (image01.shape[1], image01.shape[0]))

# Display the two original images
cv.imshow('First Image', image01)
cv.imshow('Second Image', image02)

try:
    # Ask the user for an alpha value between 0 and 1
    alpha = float(input("Enter alpha (a number between 0 and 1): "))

    # Make sure the alpha value is within the correct range
    if alpha < 0 or alpha > 1:
        raise ValueError()
except ValueError:
    # Ensures that in any of the error cases, we close the cv resources
    cv.destroyAllWindows()
    raise ValueError("Invalid input. Please enter a valid number (a value between 0 and 1) next time you run this cell.")

# Perform the linear blend of two images using OpenCV's addWeighted function
# blend = (1 - alpha) * image1 + alpha * image2
blended_image = cv.addWeighted(
    image01,      # First image (source image 1)
    1 - alpha, # Weight of the first image (1 - alpha). This determines how much of image01 is visible.
    image02,      # Second image (source image 2)
    alpha,     # Weight of the second image (alpha). This determines how much of img2 is visible.
    0          # Scalar added to each sum of the weighted pixels. Here it's 0, meaning no additional value is added.
)

# Display the blended image
cv.imshow('Blended Image', blended_image)

# Save the blended image (optional)
cv.imwrite(f'{folder_name}/blended_image.jpg', blended_image)

# Wait for a key press to close the windows
cv.waitKey(0)
cv.destroyAllWindows()


## Part 3: A Drawing Application

### Part 3: 1.1	Create a program to draw green rectangles on a image with thickness is 4. 

In [1]:
### Part 3: 1.1 Create a program to draw green rectangles on an image with thickness is 4. 
import cv2 as cv

# Get the image
folder_name = 'part3'
image03_path = f'{folder_name}/blended_image.jpg'
image03 = cv.imread(image03_path)

# Get the image's height and width
height, width = image03.shape[:2]

# Set thickness and colour for both rectangles
thickness = 4
colour = (0, 255, 0)

# Define the top-left and bottom-right coordinates of both rectangles
rect_1_top_left = (0,0)
rect_1_btm_right = (width - 1, height - 1)
rect_2_top_left = (int(width * 0.27), int(height * 0.66))
rect_2_btm_right = (int(width * 0.435), int(height * 0.80))

rectangle_1 = cv.rectangle(image03, rect_1_top_left, rect_1_btm_right, colour, thickness)
rectangle_2 = cv.rectangle(rectangle_1, rect_2_top_left, rect_2_btm_right, colour, thickness)

# Display the image
cv.imshow('image_03', rectangle_2)

cv.waitKey(0)
cv.destroyAllWindows()

### Part 3: 1.2	 Change thickness from the program made in [1.1](#11create-a-program-to-draw-green-rectangles-on-a-image-with-thickness-is-4) to -1

In [2]:
### Part 3: 1.2	Change thickness from the program made in 1.1 to -1
import cv2 as cv

# Get the image
folder_name = 'part3'
image03_path = f'{folder_name}/blended_image.jpg'
image03 = cv.imread(image03_path)

# Get the image's height and width
height, width = image03.shape[:2]

rect_top_left = (0, 0)
rect_btm_right = (width - 1, height - 1)

img_rect = cv.rectangle(image03, rect_top_left, rect_btm_right, (0, 255, 0), thickness=-1)

# Display the image
cv.imshow('image_03', img_rect)

cv.waitKey(0)
cv.destroyAllWindows()

### Part 3: 1.3 Create a program to write Text on the Rectangle in the Image

In [7]:
### Part 3: 1.3 Create a program to write Text on the Rectangle in the Image
import cv2 as cv

# Get the image
folder_name = 'part3'
image03_path = f'{folder_name}/blended_image.jpg'
image03 = cv.imread(image03_path)

# Text configuration
text = "Boat with people"
font = cv.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)  # White text
thickness = 2

# Not using the other tuple value, `baseline`. So, we use _ to denote we're not using it
(text_width, text_height), _ = cv.getTextSize(text, font, font_scale, thickness)

# Get the image's height and width
height, width = image03.shape[:2]

# Configure the rectangle's values
top_left_rect_x = int(width * 0.27)
top_left_rect_y = int(height * 0.66)
rect_top_left = (top_left_rect_x, top_left_rect_y)
rect_btm_right = (int(width * 0.435), int(height * 0.80))

# Draw the first rectangle
img_rect = cv.rectangle(image03, rect_top_left, rect_btm_right, (0, 255, 0), thickness=4)

# Draw the text on the image
img_text = cv.putText(img_rect, text, (top_left_rect_x, top_left_rect_y - text_height), font, font_scale, font_color, thickness)

# Configure the second rectangle around the text
text_rect_top_left = (top_left_rect_x - 10, top_left_rect_y - text_height - 27)  # Extra padding
text_rect_btm_right = (top_left_rect_x + text_width + 10, top_left_rect_y - 9)  # Extra padding

# Draw the rectangle surrounding the text
img_text_rect = cv.rectangle(img_text, text_rect_top_left, text_rect_btm_right, (0, 0, 255), thickness=2)  # Red rectangle

# Display the image
cv.imshow('image_03', img_text_rect)

cv.waitKey(0)
cv.destroyAllWindows()